In [1]:
#! git clone https://github.com/kumarankit1996/Dataset.git

In [2]:
import re
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import wordnet
from sklearn.feature_extraction.text import TfidfVectorizer

In [3]:
#! ls Dataset/HackerEarth/hm_train.csv

In [4]:
import numpy as np
import pandas as pd
import math
import io
from keras.layers import concatenate
import random
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dropout,UpSampling2D, Dense, MaxPooling2D, BatchNormalization, Input, Flatten, Lambda
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.layers import Dense
from keras import optimizers
import os
import math
from sklearn.utils import shuffle
import itertools
from keras.callbacks import ModelCheckpoint, EarlyStopping
#from google.colab import files
from keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn import preprocessing, model_selection
from keras.callbacks import ModelCheckpoint, EarlyStopping

Using TensorFlow backend.


In [5]:
import nltk
nltk.download('popular')

[nltk_data] Downloading collection u'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     /Users/ankit/nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     /Users/ankit/nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     /Users/ankit/nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     /Users/ankit/nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     /Users/ankit/nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     /Users/ankit/nltk_data...
[nltk_data]    |   Package movie_reviews is already up-t

True

# Reading the Dataset

In [6]:
train_data=pd.read_csv("hm_train.csv")
test_data=pd.read_csv("hm_test.csv")
#train_data['cleaned_hm'][60320],test_data['cleaned_hm'][0]

In [7]:
# Combining the train and test dataset for creating bag of words 
bigdata = pd.concat([train_data['cleaned_hm'], test_data['cleaned_hm']], ignore_index=True)
#bigdata[60320],bigdata[60321]

# Preprocessing the data

In [8]:
def clean_sentence(sentence):
    # extract the alphabet from sentences
    sentence = re.sub("[^A-Za-z]", " ", sentence)
    # changing the case of sentence to lower case
    sentence = sentence.lower()
    # tokenizing words in sentences
    sentence = word_tokenize(sentence)
    # Converting to bi grams
    #sentence=nltk.bigrams(sentence)
    # defined the word lemmatizer object
    lemma = WordNetLemmatizer()
    # deleting the stop words and finding the lemmatize words
    sentence = [lemma.lemmatize(word,wordnet.VERB) for word in sentence if word not in set(stopwords.words("english"))]
    # joining the words to form sentence
    sentence = " ".join(sentence)
    return sentence

In [9]:
# Converting the target into one hot vector
def category(data,s="NN"):
    target=[]
    for i in range(len(data)):
        if data[i]=='bonding':
            target.append(0)
        elif data[i]=='achievement':
            target.append(1)
        elif data[i]=='affection':
            target.append(2)
        elif data[i]=='leisure':
            target.append(3)
        elif data[i]=='enjoy_the_moment':
            target.append(4)
        elif data[i]=='nature':
            target.append(5)
        elif data[i]=='exercise':
            target.append(6)
    if s=="NN":
        target = to_categorical(target)
    return target

In [10]:
# Cleaning the dataset
x=[]
for i in range(len(bigdata)):
    x.append(clean_sentence(bigdata[i]))

In [11]:
# Counting the frequency of words
count_vectorizer = CountVectorizer()
C = count_vectorizer.fit_transform(x).toarray()

In [12]:
#tfidf_vectorizer = TfidfVectorizer()
#values = tfidf_vectorizer.fit_transform(x).toarray()

# Creating the train and test set from bag of words
X=C[:60321]
test_x=C[60321:]
test_x.shape,X.shape

((40213, 19802), (60321, 19802))

In [13]:
# Converting the target into one hot vector
Y=category(train_data['predicted_category'])

In [14]:
test_x.shape

(40213, 19802)

# Neural Network Model

In [38]:
mc = ModelCheckpoint(filepath='2weights_v8.h5', monitor='val_loss',period=1,save_best_only=True,save_weights_only=True,mode='auto', verbose = 3)
es = EarlyStopping(patience=1000, monitor='val_loss', min_delta=0.0005, mode='auto')

In [39]:
input_dim = len(X[0])

model = Sequential()
model.add(Dense(100, input_dim = input_dim , activation = 'relu'))
model.add(Dense(150, activation = 'relu'))
model.add(Dense(150, activation = 'relu'))
model.add(Dense(200, activation = 'relu'))
model.add(Dense(200, activation = 'relu'))
model.add(Dense(7, activation = 'softmax'))

model.compile(loss = 'categorical_crossentropy' , optimizer =optimizers.Adadelta(lr=0.01) , metrics = ['accuracy'] )

In [ ]:
model.fit(X, Y, epochs = 200,callbacks=[mc,es], batch_size = 64,validation_split=0.2)

In [41]:
model.load_weights('2weights_v8.h5')

# Prediction

In [42]:
predicted = model.predict(test_x)

In [43]:
p=np.argmax(predicted,axis=1)

In [44]:
predict=[]
for i in range(len(p)):
    if p[i]==0:
        predict.append('bonding')
    elif p[i]==1:
        predict.append('achievement')
    elif p[i]==2:
        predict.append('affection')
    elif p[i]==3:
        predict.append('leisure')
    elif p[i]==4:
        predict.append('enjoy_the_moment')
    elif p[i]==5:
        predict.append('nature')
    elif p[i]==6:
        predict.append('exercise')

In [45]:
predict=np.array(predict).reshape(-1,1)
uid=np.array(test_data['hmid']).reshape(-1,1)
output=np.hstack((uid,predict))

In [47]:
output=pd.DataFrame(output)
output.columns=['hmid','predicted_category']
output.to_csv("output22.csv",sep=",",index=False)

# SVM Model

In [ ]:
Y=category(train_data['predicted_category'],"SVM")

In [17]:
train_x, test_x, train_y, test_y = model_selection.train_test_split(X,Y,test_size = 0.1, random_state = 0)

In [ ]:
from sklearn.svm import SVC
# defining the SVM Model
SVM_model=SVC(kernel='rbf',C=2,gamma=0.05)
history=SVM_model.fit(X,Y)

In [ ]:
#Calculate Erms 
def ERMS(Predicted,Actual):
    total=0.0
    counter=0
    for i in range(0,len(Predicted)):
        total+=math.pow((Actual[i]-Predicted[i]),2)
        #count for match
        if (int(np.around(Predicted[i])) == Actual[i]):
            counter+=1
    Accuracy=(counter*100/len(Predicted))
    #print(counter)
    Erms=math.sqrt(total/len(Predicted))
    return Erms, Accuracy

In [ ]:
SVM_PredictedTarget=SVM_model.predict(test_x)
#Erms, SVM_Accu=ERMS(SVM_PredictedTarget,test_y)
#print("Erms:",Erms)
#print("SVM USPS Accu:",SVM_Accu)

# Prediction

In [ ]:
p=SVM_PredictedTarget
predict=[]
for i in range(len(p)):
    if p[i]==0:
        predict.append('bonding')
    elif p[i]==1:
        predict.append('achievement')
    elif p[i]==2:
        predict.append('affection')
    elif p[i]==3:
        predict.append('leisure')
    elif p[i]==4:
        predict.append('enjoy_the_moment')
    elif p[i]==5:
        predict.append('nature')
    elif p[i]==6:
        predict.append('exercise')

In [ ]:
predict=np.array(predict).reshape(-1,1)
uid=np.array(test_data['hmid']).reshape(-1,1)
output=np.hstack((uid,predict))
output=pd.DataFrame(output)
output.columns=['hmid','cleaned_hm']
output.to_csv("svmoutput2.csv",sep=",",index=False)